In [122]:
import pandas as pd
import datetime 
from pandas.tseries.offsets import BDay

In [123]:
price_df = pd.read_csv(
    'blmn_close_prices.csv',
    infer_datetime_format=True,
    parse_dates=True)
    #index_col='time')

price_df['date'] = price_df['time']
#
del price_df['time']
#
##price_df.iloc[1]
date = price_df.pop('date')
price_df.insert(0, 'date', date)
#
##price_df = price_df.rename(columns={'time':'date'})
#price_df.set_index('date')
price_df.date = pd.to_datetime(price_df.date, errors='coerce', utc=True).dt.date

price_df.head()
#price_df.info()

,date,BLMN
0,2021-01-04,18.340
1,2021-01-05,19.050
2,2021-01-06,20.310
3,2021-01-07,20.560
4,2021-01-08,20.675


In [124]:
weather_df = pd.read_csv(
    'data/cleaned_data/df_weather_data_updated.csv',
    infer_datetime_format=True,
    parse_dates=True)

#isBusinessDay = BDay().onOffset
#match_series = pd.to_datetime(weather_df['date']).map(isBusinessDay)
#match_series
#weather_df[match_series]

del weather_df['Unnamed: 0']
del weather_df['Unnamed: 0.1']
del weather_df['Unnamed: 0.1.1']

weather_df.date = pd.to_datetime(weather_df.date, errors='coerce', utc=True).dt.date

weather_df.head()

,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,location.timezone_id,...,dewpoint,windchill,windgust,feelslike,date,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,Zipcode,34231,en,m,Sarasota,USA,Florida,27.272,-82.517,America/New_York,...,18,21,45,21,2021-01-01,20,23,21,0.0,8.7
1,Zipcode,34201,en,m,Bradenton,USA,Florida,27.403,-82.477,America/New_York,...,18,22,41,23,2021-01-01,20,24,22,0.0,8.7
2,Zipcode,33710,en,m,Saint Petersburg,USA,Florida,27.789,-82.729,America/Kentucky/Monticello,...,17,20,51,20,2021-01-01,20,21,21,0.0,8.7
3,Zipcode,34994,en,m,Stuart,USA,Florida,27.198,-80.253,America/New_York,...,19,24,31,26,2021-01-01,23,26,25,0.0,8.7
4,Zipcode,32309,en,m,Tallahassee,USA,Florida,30.597,-84.042,America/New_York,...,18,20,27,20,2021-01-01,18,23,21,0.0,3.8


In [125]:
weather_df_sliced = weather_df.drop(columns=[
    'request.type', 
    'request.language', 
    'request.unit',
    'location.lat',
    'location.lon',
    'location.timezone_id',
    'location.utc_offset',
    ])

weather_df_sliced = weather_df_sliced.drop(weather_df_sliced.columns[[0,1]], axis=1)

weather_df_sliced.rename(columns={
    'location.region':'state', 
    'request.query': 'zipcode', 
    'location.name': 'city',
    'location.country':'country'
    }, inplace=True)

date = weather_df_sliced.pop('date')
weather_df_sliced.insert(0, 'date', date)

state = weather_df_sliced.pop('state')
weather_df_sliced.insert(1, 'state', state)

weather_df_sliced.tail()

,date,state,country,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
109599,2021-03-16,Florida,USA,29,0.0,64,10,1018,8,24,15,23,21,24,19,29,24,0.0,11.6
109600,2021-03-16,Florida,USA,29,0.0,64,10,1018,7,24,16,24,19,24,19,29,25,0.0,11.6
109601,2021-03-16,Florida,USA,24,0.0,80,10,1018,15,23,18,22,26,23,19,24,22,0.0,11.6
109602,2021-03-16,Georgia,USA,18,36.7,84,8,1014,93,15,12,14,14,14,11,18,16,0.0,6.0
109603,2021-03-16,Florida,USA,24,0.0,73,10,1019,24,25,18,23,33,25,22,24,23,0.0,11.6


In [126]:
Arizona = weather_df_sliced[weather_df_sliced['state']=='Arizona'].groupby(['state', 'date'], as_index=False).mean()
Florida = weather_df_sliced[weather_df_sliced['state']=='Florida'].groupby(['state', 'date'], as_index=False).mean()
Georgia = weather_df_sliced[weather_df_sliced['state']=='Georgia'].groupby(['state', 'date'], as_index=False).mean()
Nevada = weather_df_sliced[weather_df_sliced['state']=='Nevada'].groupby(['state', 'date'], as_index=False).mean()
South_Carolina = weather_df_sliced[weather_df_sliced['state']=='South Carolina'].groupby(['state', 'date'], as_index=False).mean()
#Arizona.set_index('date')

In [127]:
arizona_df = pd.merge(price_df, Arizona, how='inner', on='date')

arizona_df['BLMN'] = price_df['BLMN']

arizona_df.head()

,date,BLMN,state,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,2021-01-04,18.340,Arizona,17.733333,0.0,31.533333,10.0,1019.066667,8.266667,13.133333,-3.933333,12.666667,7.133333,12.666667,8.733333,17.733333,13.066667,0.0,8.700000
1,2021-01-05,19.050,Arizona,20.133333,0.0,31.933333,10.0,1017.933333,12.400000,15.066667,-2.200000,14.866667,7.200000,14.866667,10.933333,20.133333,15.800000,0.0,8.700000
2,2021-01-06,20.310,Arizona,18.047619,0.0,28.619048,10.0,1019.523810,23.523810,13.904762,-4.857143,13.000000,11.380952,13.000000,9.285714,18.047619,13.952381,0.0,7.242857
3,2021-01-07,20.560,Arizona,20.133333,0.0,22.466667,10.0,1018.133333,25.533333,14.400000,-7.266667,14.266667,6.933333,14.266667,10.600000,20.133333,15.533333,0.0,8.700000
4,2021-01-08,20.675,Arizona,18.800000,0.0,22.866667,10.0,1018.133333,41.266667,14.400000,-7.000000,14.333333,6.733333,14.333333,10.666667,18.800000,15.200000,0.0,6.853333


In [128]:
florida_df = pd.merge(price_df, Florida, how='right', on='date')

florida_df['BLMN'] = price_df['BLMN']

florida_df.head()

,date,BLMN,state,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,2021-01-01,18.340,Florida,25.148325,1.898086,76.851675,9.784689,1019.956938,33.287081,24.062201,18.263158,22.736842,34.047847,23.837321,20.712919,25.148325,23.157895,0.0,7.985646
1,2021-01-02,19.050,Florida,24.569378,3.176555,79.947368,9.641148,1019.172249,49.722488,23.497608,18.531100,22.138756,29.052632,23.267943,20.339713,24.569378,22.588517,0.0,6.640191
2,2021-01-03,20.310,Florida,21.870813,1.005263,77.090909,9.401914,1017.425837,52.961722,20.918660,15.966507,20.129187,24.827751,20.641148,17.889952,21.870813,20.483254,0.0,5.852153
3,2021-01-04,20.560,Florida,18.531100,0.124402,60.258373,9.966507,1018.114833,28.937799,17.062201,8.641148,16.239234,17.559809,16.377990,14.363636,18.531100,16.593301,0.0,7.513397
4,2021-01-05,20.675,Florida,18.732057,0.000000,55.483254,10.000000,1018.382775,0.669856,15.473684,6.282297,15.071770,18.449761,15.071770,12.444976,18.732057,15.866029,0.0,8.700000


In [129]:
georgia_df = pd.merge(price_df, Georgia, how='right', on='date')

georgia_df['BLMN'] = price_df['BLMN']

georgia_df.head()


,date,BLMN,state,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,2021-01-01,18.340,Georgia,19.354839,35.009677,95.322581,6.903226,1017.161290,94.516129,16.645161,15.935484,16.000000,25.258065,16.000000,11.290323,19.354839,17.451613,0.0,3.987097
1,2021-01-02,19.050,Georgia,17.064516,9.838710,84.387097,8.354839,1016.032258,60.483871,15.451613,12.677419,15.290323,11.870968,15.290323,13.000000,17.064516,15.096774,0.0,7.322581
2,2021-01-03,20.310,Georgia,10.806452,10.403226,76.032258,9.516129,1015.806452,47.709677,10.000000,5.806452,7.838710,19.806452,7.838710,7.032258,10.806452,9.451613,0.0,7.090323
3,2021-01-04,20.560,Georgia,12.032258,0.000000,65.935484,10.000000,1019.129032,9.225806,7.354839,1.096774,5.322581,15.322581,5.322581,3.741935,12.032258,7.870968,0.0,8.700000
4,2021-01-05,20.675,Georgia,11.806452,0.070968,68.709677,9.645161,1018.032258,28.516129,8.677419,2.645161,6.774194,16.741935,6.774194,6.193548,11.806452,9.064516,0.0,7.732258


In [130]:
nevada_df = pd.merge(price_df, Nevada, how='right', on='date')

nevada_df['BLMN'] = price_df['BLMN']

nevada_df.head()

,date,BLMN,state,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,2021-01-01,18.340,Nevada,8.250,0.0000,49.875,10.0,1021.500,8.750,4.500,-6.250,1.625,25.500,1.625,0.375,8.250,5.000,0.000,8.7
1,2021-01-02,19.050,Nevada,9.500,0.0000,53.375,10.0,1023.250,21.875,4.500,-5.375,3.625,6.500,3.625,-0.750,9.500,5.000,0.000,8.7
2,2021-01-03,20.310,Nevada,9.875,0.0250,52.875,10.0,1022.000,18.625,4.875,-5.000,4.375,6.875,4.375,-0.250,9.875,5.875,0.000,8.7
3,2021-01-04,20.560,Nevada,11.625,2.2375,46.250,9.5,1018.000,29.375,7.250,-5.000,6.250,11.375,6.250,2.375,11.625,8.000,1.225,7.4
4,2021-01-05,20.675,Nevada,12.750,0.5000,47.625,10.0,1021.125,9.875,7.625,-4.625,6.500,12.500,6.500,2.125,12.750,8.125,0.000,8.7


In [131]:
south_carolina_df = pd.merge(price_df, South_Carolina, how='right', on='date')

south_carolina_df['BLMN'] = price_df['BLMN']

south_carolina_df.head()

,date,BLMN,state,temperature,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,mintemp,maxtemp,avgtemp,totalsnow,sunhour
0,2021-01-01,18.340,South Carolina,19.541667,17.779167,94.041667,7.291667,1020.375000,87.916667,17.333333,16.333333,16.708333,23.041667,16.708333,13.625000,19.541667,17.875000,0.0,3.933333
1,2021-01-02,19.050,South Carolina,19.208333,18.241667,89.250000,8.833333,1016.083333,72.958333,17.666667,15.958333,17.375000,24.291667,17.375000,14.250000,19.208333,18.000000,0.0,5.591667
2,2021-01-03,20.310,South Carolina,16.583333,17.937500,79.708333,8.625000,1012.708333,48.000000,15.000000,11.458333,14.250000,25.666667,14.250000,10.875000,16.583333,15.208333,0.0,7.470833
3,2021-01-04,20.560,South Carolina,12.875000,0.000000,67.291667,10.000000,1017.041667,21.708333,9.666667,3.750000,8.583333,12.916667,8.583333,7.083333,12.875000,10.166667,0.0,8.287500
4,2021-01-05,20.675,South Carolina,13.333333,0.008333,66.541667,9.958333,1015.791667,23.125000,9.333333,3.000000,7.750000,17.208333,7.750000,6.291667,13.333333,9.833333,0.0,8.300000
